In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove

In [44]:
# The smart contract as a transition system.
ts = Ts('auction')

goal = z3.BitVec('goal',256)
closeTime = z3.BitVec('closeTime', 256)
timestamp = z3.BitVec('now', 256)

addrSort = z3.BitVecSort(256)

deposits, depositsOut = ts.add_var(z3.ArraySort(addrSort, z3.BitVecSort(256)),
        name= 'deposits')
totalDeposits, totalDepositsOut = ts.add_var(z3.BitVecSort(256), name='totalDeposits')

raised, raisedOut = ts.add_var(z3.BitVecSort(256), name='raised')

closed, closedOut = ts.add_var(z3.BoolSort(), name='closed')

In [47]:
p = z3.BitVec('p',256)

ts.Init = z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,    
    closed == False,
    totalDeposits == 0
)

amount = z3.BitVec('amount',256)

tr_invest = z3.And(
    timestamp <= closeTime,
    raised < goal,
    raisedOut == raised + amount,
    depositsOut == z3.Update(deposits,p,deposits[p]+amount),
    totalDepositsOut == totalDeposits+amount
)

tr_close = z3.And(
    timestamp > closeTime,
    depositsOut == deposits,
    raisedOut == raised,
    totalDepositsOut == totalDeposits
)

tr_refund = z3.And(
    closed,
    raised < goal,
    deposits[p] > 0,
    depositsOut == z3.Update(deposits,p,0),
    raisedOut == raised - deposits[p],
    totalDepositsOut == totalDeposits - deposits[p]
)

tr_withdraw = z3.And(
    closed,
    raised > goal,
    raisedOut == 0,
    z3.ForAll(p, deposits[p]==0),
    totalDepositsOut == 0
)

ts.Tr = z3.Or(tr_invest, tr_close, tr_refund, tr_withdraw)

In [48]:
ts_property = raised == totalDeposits

In [49]:
s = prove_inductive(ts, ts_property)

Prove init => property.
proved
Prove property is inductive.
proved
